In [ ]:
def trace(root):
    node = []
    


def draw_dot(root):

    nodes = 



In [3]:
from queue import Queue
class Value:

    def __init__(self, data, children = (), _op = "", label = ""):
        self.data = data
        self._next = set(children)
        self._op =  _op
        self.label = label
        self.grad = 0
        self.backward = lambda: None


    def __add__(self, other):
        other = other if isinstance(other, Value) else Value(other)

        out = Value(self.data + other.data, (self, other),"+")
        def _backward():

            self.grad += 1*out.grad
            other.grad += 1*out.grad

        out.backward = _backward

        return out
    
    def __mul__(self, other):
        other = other if isinstance(other, Value) else Value(other)

        out = Value(self.data + other.data, (self, other),"+")
        def _backward(self, other):

            self.grad = other.grad * out.grad
            other.grad = self.grad * out.grad

        out.backward = _backward
        
        return out
    
    def backward(self):
        nodes = [self]

        while nodes:
            current = nodes.pop()
            for n in current._prev:
                nodes.append(n)

            current.backward()
    
    def __repr__(self) -> str:
        return f"Value(data= {self.data})"
    



In [5]:
a = Value(2)
b = Value(-3)
c = Value(10)

d = a*b
e = d + c

f = Value(-2)

L = d*f

In [ ]:
L.backward()